# Pràctica 4
### Part I : Entrenament de models Word2vec

In [1]:
from datasets import load_dataset
from nltk.tokenize import word_tokenize

dataset = load_dataset("projecte-aina/catalan_general_crawling")

Generating train split: 0 examples [00:00, ? examples/s]

Dataset catalan_general_crawling downloaded and prepared to C:/Users/adria/.cache/huggingface/datasets/projecte-aina___catalan_general_crawling/default/1.0.0/fb8a4dbf3849d7aad584ad030c058c67c6a935ee7b8d6e37411514da9376a2fb. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset['train']


Dataset({
    features: ['text'],
    num_rows: 1016113
})

In [3]:
from gensim.models import word2vec
model = word2vec.Word2Vec(dataset["train"]["text"] , vector_size=100, window=5, min_count=10, workers=4, epochs=25, sg=1)


KeyboardInterrupt: 

## Text similarity training and comparison

### Obtain the Baseline model

### Compare results with different word embeddings

#### 1. One Hot encoding

In [ ]:
# limitar mida?, normalitzar?

#### 2. Models de Word2Vec/GloVe pre-entrenats

##### 2.1.Word2Vec + Mean

##### 2.2.Word2Vec + Mean ponderada (TF-IDF)

#### 3.Spacy

#### 4.RoBERTa

##### 4.1.CLS

In [ ]:
# (Amb spaCy, doc._.trf_data.tensors[-1])

##### 4.2.MEAN

In [ ]:
# (Amb spaCy, doc._.trf_data.tensors[-1])

#### 5.RoBERTa FineTuned

## Train the same model with initiated trainable embeddings

#### Random Embeddings (uniforme)

#### Word2Vec

## Analyze results